# VacationPy
----


In [1]:
# dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# format input filename
input_file = "output_data/cities.csv"

# read weather data into a dataframe
weather_df = pd.read_csv(input_file)

# display a sample of the dataframe
weather_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,69.80,78,1,6.91,US,1610090621
1,Mareeba,-17.0000,145.4333,86.00,79,75,4.61,AU,1610090246
2,Cabo San Lucas,22.8909,-109.9124,61.00,68,0,1.01,MX,1610090457
3,Saskylakh,71.9167,114.0833,-25.15,85,99,6.76,RU,1610090151
4,Cross Lanes,38.4204,-81.7907,35.01,80,90,6.91,US,1610090458
5,East London,-33.0153,27.9116,73.40,78,20,17.27,ZA,1610090190
6,Hobart,-42.8794,147.3294,69.01,63,20,17.27,AU,1610089986
7,Gasa,31.5000,34.4667,66.99,35,0,1.99,PS,1610090458
8,Cape Town,-33.9258,18.4232,75.00,53,0,18.41,ZA,1610090281
9,Awjilah,29.1081,21.2869,61.61,48,14,1.86,LY,1610090459


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

# store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# pull a list of humidity data
humidity = weather_df["Humidity"].astype(float)

# create a heatmap weighted on humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                point_radius = 2)

# configure the display of the map
fig = gmaps.figure(center=(10,0.1), zoom_level=1.8)

# add the heatmap  layer to our figure
fig.add_layer(heat_layer)

# display the map
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# key on weather variables to narrow down our city selection
vacay_df = weather_df.loc[(weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 85) & (weather_df["Humidity"] < 60) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] < 70)]

# drop any rows with null data
hotel_df = vacay_df.dropna(how="any").copy()

# display resultant dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,Tahoua,14.8888,5.2692,71.60,17,0,6.91,NE,1610090461
101,Tezu,27.9167,96.1667,80.19,23,0,4.00,IN,1610090479
158,Antanifotsy,-19.6500,47.3167,78.17,48,0,3.02,MG,1610090493
178,Maridi,4.9158,29.4742,83.61,26,0,6.67,SS,1610090498
196,Wendo,6.6000,38.4167,70.48,38,67,3.36,ET,1610090504
213,Bilma,18.6853,12.9164,72.01,21,0,7.31,NE,1610090265
285,Salalah,17.0151,54.0924,78.80,22,0,9.22,OM,1610090028
305,Barwadih,23.8333,84.1167,83.41,33,2,4.05,IN,1610090535
312,Along,28.1667,94.7667,77.16,35,0,1.97,IN,1610090218
409,Tonj,7.2737,28.6822,83.08,25,0,5.97,SS,1610090212


###### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# insert a Hotel Name column to our dataframe
# with a default value of empty string
temp_hotelname = [""] * len(hotel_df.index)
hotel_df.insert(9, "Hotel Name", temp_hotelname, True)

# display the updated dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,Tahoua,14.8888,5.2692,71.60,17,0,6.91,NE,1610090461,
101,Tezu,27.9167,96.1667,80.19,23,0,4.00,IN,1610090479,
158,Antanifotsy,-19.6500,47.3167,78.17,48,0,3.02,MG,1610090493,
178,Maridi,4.9158,29.4742,83.61,26,0,6.67,SS,1610090498,
196,Wendo,6.6000,38.4167,70.48,38,67,3.36,ET,1610090504,
213,Bilma,18.6853,12.9164,72.01,21,0,7.31,NE,1610090265,
285,Salalah,17.0151,54.0924,78.80,22,0,9.22,OM,1610090028,
305,Barwadih,23.8333,84.1167,83.41,33,2,4.05,IN,1610090535,
312,Along,28.1667,94.7667,77.16,35,0,1.97,IN,1610090218,
409,Tonj,7.2737,28.6822,83.08,25,0,5.97,SS,1610090212,


In [10]:
# find a hotel near the city's coordinates

# format the URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# looking for lodging type locations within 5000 meters of the city
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# create empty list for hotel coordinates
coordinates = []

# loop through qualified dataframe
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lon = row['Lng']
    city = row["City"]

    # update location key value based on city
    # coordinates
    params["location"] = f"{lat},{lon}"

    # make request
    try:
        hotel_hit = requests.get(base_url, params=params).json()
        
        # grab hotel coordinates and add to list
        coordinates.append((hotel_hit["results"][0]["geometry"]["location"]["lat"], hotel_hit["results"][0]["geometry"]["location"]["lng"]))
        
        # add hotel name to dataframe
        hotel_df.loc[index, "Hotel Name"] = hotel_hit["results"][0]["name"]
    except:
        # location probably remote: no hotels within 5000 meters
        print(f"No hotels found within 5000 meters of: {city}")


No hotels found within 5000 meters of: Antanifotsy
No hotels found within 5000 meters of: Maridi
No hotels found within 5000 meters of: Tonj


In [11]:
# remove rows where we could not find a hotel and
# display updated dataframe
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,Tahoua,14.8888,5.2692,71.60,17,0,6.91,NE,1610090461,Bungalows Guesthouse
101,Tezu,27.9167,96.1667,80.19,23,0,4.00,IN,1610090479,Circuit House
196,Wendo,6.6000,38.4167,70.48,38,67,3.36,ET,1610090504,Bekure tseon hotal
213,Bilma,18.6853,12.9164,72.01,21,0,7.31,NE,1610090265,Expeditions Ténére voyages
285,Salalah,17.0151,54.0924,78.80,22,0,9.22,OM,1610090028,HAMDAN PLAZA HOTEL
305,Barwadih,23.8333,84.1167,83.41,33,2,4.05,IN,1610090535,Forest Rest House
312,Along,28.1667,94.7667,77.16,35,0,1.97,IN,1610090218,Hotel West
424,Mecca,21.4267,39.8261,77.61,47,0,2.82,SA,1610090762,"Makkah Clock Royal Tower, A Fairmont Hotel"
427,Tikapur,28.5000,81.1333,79.03,24,0,6.46,NP,1610090564,BANANA AGRO RESORT


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates, info_box_content=hotel_info)

# add the layer to the map
fig.add_layer(markers)

# display figure
fig


Figure(layout=FigureLayout(height='420px'))